# Vehicle detection

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Manual vehicle detection

In [2]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    draw_img = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
    return draw_img

image = mpimg.imread('test_images/1.jpg')
bboxes = [((250, 500), (400, 600)), ((800, 500), (1200, 700))]
draw_img = draw_boxes(image, bboxes)

f, axs = plt.subplots(1, 2, figsize=(24, 9))
axs[0].imshow(image), axs[0].set_title('first', fontsize=30)
axs[1].imshow(draw_img), axs[1].set_title('after', fontsize=30)

(<matplotlib.image.AxesImage at 0x7fc028f33d68>,
 <matplotlib.text.Text at 0x7fc028f94fd0>)